In [188]:
import torch
import numpy as np
import os
import pickle as pkl
from copy import copy

In [89]:
from typing import Dict, Union

In [80]:
root_folder = "data/cardio/Results"
root_folder = "/home/rodrigo/CISTIB/UKBB/data/meshes/FHM/Results"

In [150]:
ids = os.listdir(root_folder)
ids = ids[:1000]

# Mesh generator

In [154]:
def get_3d_mesh(ids, root_folder):
    
    for id in ids:
        try:
          npy_file = f"{root_folder}/{id}/models/LV_time001.npy"
          pc  = np.load(npy_file)
          yield id, pc
        except:
          continue
        

In [155]:
meshes_gen = get_3d_mesh(ids, root_folder)
meshes = {}

for id, mesh in meshes_gen:
    meshes[id] = mesh        

___

# Apply Procrustes transforms

In [157]:
procrustes_transforms = pkl.load(open("procrustes_transforms_35k.pkl", "rb"))

In [186]:
def transform_mesh(mesh, rotation: Union[None, np.array] = None, traslation: Union[None, np.array] = None):
    mesh = copy(mesh)
    if traslation is not None:
        mesh = mesh - traslation
    if rotation is not None:
        centroid = mesh.mean(axis=0)
        mesh -= centroid
        mesh = mesh.dot(rotation)
        mesh += centroid
    return mesh

In [103]:
id = "1000215"
transform_mesh(meshes[id], **procrustes_transforms[id])

array([[ 24.65671847,   4.0194862 ,  27.39147769],
       [ 10.94219057,   1.82538719, -52.70756595],
       [ 29.76329085, -11.48212941,  20.35052923],
       ...,
       [-25.35006027,  -0.80063413,   6.69747019],
       [ 32.20795795,   7.10482452,  -1.91620522],
       [  9.30463436,  -2.79676522, -52.40960088]])

___

# Display transformed meshes

In [159]:
import ipywidgets as widgets
from ipywidgets import interact
import pyvista as pv
import random

In [190]:
# sphere = vedo.Sphere(res=params["mesh_resolution"]).to_trimesh()
# conn = sphere.faces # connectivity
# conn = np.c_[np.ones(conn.shape[0]) * 3, conn].astype(int)  # add column of 3, as required by PyVista

pv.set_plot_theme("document")

faces, _ = pkl.load(open("faces_and_downsampling_mtx_frac_0.1_LV.pkl", "rb")).values()
faces = np.c_[np.ones(faces.shape[0]) * 3, faces].astype(int)

color_palette = list(pv.colors.color_names.values())
random.shuffle(color_palette)


def f(ids, rotated, traslated):
    
    
        
    pl = pv.Plotter(notebook=True, off_screen=False, polygon_smoothing=False)
    
    for i, id in enumerate(ids):
      
      mesh = meshes[id]
      if rotated and not traslated:
        mesh = transform_mesh(mesh, rotation=procrustes_transforms[id]["rotation"])
      elif traslated and not rotated:
        mesh = transform_mesh(mesh, traslation=procrustes_transforms[id]["traslation"])
      elif traslated and rotated:
        mesh = transform_mesh(mesh, **procrustes_transforms[id])
      mesh = pv.PolyData(mesh, faces)
        
      pl.add_mesh(mesh, show_edges=False, point_size=1.5, color=color_palette[i], opacity=0.5)
    pl.show(interactive=True, interactive_update=True)
    
#interact(f, i=widgets.IntSlider(min=1,max=N))
interact(f, 
  ids=widgets.SelectMultiple(options=ids),
  rotated=widgets.ToggleButton(),
  traslated=widgets.ToggleButton()
);

A Jupyter Widget